# Introducing text information signals into CatBoost

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/text_features/text_features_in_catboost.ipynb)

## Task description

There is a large number of tasks associated with the analysis of textual information. An example is such well-known problems as: spam or not spam email classification, analyzing the tonality of the text, dialog systems, etc. 

Sometimes the original task can be reduced to the simple ML **classification task** on a given set of observations $D:$<br> $D = \{(x_i, c_i)\}$, $x_i = (x_{i,0}, x_{i,1}, ..., x_{i,m})$ is features vector and $c_i$ is a class of $i$-th object.<br>
In such task the features may contain not only $x_{i,j}$ as numeric or categorical values, but also the source text (e.g. tweet or question).

Here and further by **Text** we imply a sequence of symbols $(a_0, a_1, ..., a_k), a_i \in A$,<br> $A$ called as alphabet, e.g. $A$ can be a set of English letters, Unicode symbols or $A$ can have a more complicated structure including a sequences of symbols, also called tokens, e.g. $A$ can be a set of English words or Emoji.

## Example: Rotten tomatoes dataset

Let's download real dataset for examples with CatBoost.
This one is based on rotten tomatoes movie reviews data source and was taken from kaggle: https://www.kaggle.com/rpnuser8182/rotten-tomatoes <br>

Dataset contains features about movie: description, MPAA rating, producer, company, etc, and features about user review: user name, review, rating,...  
Here we set the task to predict the rating of the film according to the user's review

In [2]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostClassifier
from catboost.datasets import rotten_tomatoes

In [3]:
learn, _ = rotten_tomatoes()
print('Feature names:\n' + ', '.join(list(learn)))

Feature names:
id, synopsis, rating_MPAA, genre, director, writer, theater_date, dvd_date, box_office, runtime, studio, dvd_date_int, theater_date_int, review, rating, fresh, critic, top_critic, publisher, date, date_int, rating_10


In [4]:
learn.head(5)

,id,synopsis,rating_MPAA,genre,director,writer,theater_date,dvd_date,box_office,runtime,...,theater_date_int,review,rating,fresh,critic,top_critic,publisher,date,date_int,rating_10
0,830.0,A gay New Yorker stages a marriage of convenie...,R,Art House and International | Comedy | Drama |...,Ang Lee,Ang Lee | James Schamus | Neil Peng,1993-08-04,2004-06-15,NaN,111.0,...,19930804,NaN,0.800000,fresh,Carol Cling,0,Las Vegas Review-Journal,2004-04-16,20040416.0,8.0
1,1161.0,Screenwriter Nimrod Antal makes an impressive ...,R,Action and Adventure | Art House and Internati...,NaN,NaN,2005-04-01,2005-08-30,116783.0,105.0,...,20050401,"One very long, dark ride.",0.647059,rotten,NaN,0,E! Online,2005-04-22,20050422.0,6.0
2,596.0,"""Arctic Tale"" is an epic adventure that explor...",G,Documentary | Special Interest,Adam Ravetch | Sarah Robertson,Linda Woolverton | Mose Richards | Kristin Gore,2007-08-17,2017-08-01,598103.0,86.0,...,20070817,I'm no holdout about the reality of global war...,0.625000,rotten,Jack Mathews,1,New York Daily News,2007-07-27,20070727.0,6.0
3,1585.0,A dating doctor claims that with his services ...,PG-13,Comedy | Romance,Andrew Tennant | Andy Tennant,Kevin Bisch,2005-02-11,2005-06-14,177575142.0,120.0,...,20050211,... Adds up to far more than the formula typic...,0.875000,fresh,Greg Maki,0,"Star-Democrat (Easton, MD)",2005-02-11,20050211.0,9.0
4,603.0,R&amp;B; star Janet Jackson made an impressive...,R,Drama,John Singleton,John Singleton,1993-07-23,2001-05-08,NaN,104.0,...,19930723,NaN,0.600000,fresh,Clint Morris,0,Film Threat,2005-05-06,20050506.0,6.0


### Features description 

|Id | Feature name      |  Description                                                                                 |
|---|-------------------|----------------------------------------------------------------------------------------------|
| 1 | ``id``            |  unique movie id                                                                             |
| 2 | ``synopsis``      |  brief summary of the major points of a movie                                                |
| 3 | ``rating_MPAA``   |  film rating by MPAA rating system                                                           |
| 4 | ``genre``         |  list of genres that are suitable for this film (e.g. Action, Adventure, Comedy,...          |
| 5 | ``director``      |  list of persons who direct the making of a film                                             |
| 6 | ``writer``        |  list of persons who write a screenplay                                                      |
| 7 | ``theater_date``  |  the date when film was first shown to the public in cinema (string)                         |
| 8 | ``dvd_date``      |  the date when film was released on DVD (string)                                             |
| 9 | ``box_office``    |  the amount of money raised by ticket sales (revenue)                                        |
| 10 | ``runtime``      |  film duration in minutes                                                                    |
| 11 | ``studio``       |  is a major entertainment company or motion picture company (20th Century Fox, Sony Pictures)|
| 12 | ``dvd_date_int`` |  the date when film was released on DVD (converted to integer)                               |
| 13 | ``theater_date_int`` |  the date when film was first shown to the public in cinema (converted to integer)       |
| 14 | ``review``       |  review of a movie, that was written by a critic                                             |
| 15 | ``rating``       |  float rating from 0 to 1 of the film according to the Rotten tomatoes web site              |
| 16 | ``fresh``        |  freshness of review - fresh or rotten                                                       |
| 17 | ``critic``       |  name of reviewer                                                                            |
| 18 | ``top_critic``   |  binary feature, is reviewer a top critic or not                                             |
| 19 | ``publisher``    |  journal or website where the review was published                                           |
| 20 | ``date``         |  the date when critic publish review (string)                                                |
| 21 | ``date_int``     |  the date when critic publish review (converted to integer)                                  |
| 22 | ``rating_10``    |  integer rating from 0 to 10 of the film according to the critic                             |

We mark as **auxiliary** columnns 'id' and 'rating', because they can be the reason of overfitting, 'theater_date','dvd_date','date' because we convert them into integers.

We mark as **text** features 'synopsis' because it is short *text* description of a film, 'genre' because it is combination of categories (we know that strings have structure where words define categories), for example 'Action | Comedy | Adventure', 'director' and 'writer' features are included to the text features by the same reason, 'review' becuase it is a *text* summary of critic opinion.

We mark as **categorical** features 'rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic' and 'publisher' because they can not be splitted into the group of categorical features and feature values can not be compared.

The other columns considered as **numeric**.

In [5]:
auxiliary_columns = ['id', 'theater_date', 'dvd_date', 'rating', 'date']
cat_features = ['rating_MPAA', 'studio', 'fresh', 'critic', 'top_critic', 'publisher']
text_features = ['synopsis', 'genre', 'director', 'writer', 'review']

In [6]:
def get_processed_rotten_tomatoes():
    learn, test = rotten_tomatoes()
    
    def fill_na(df, features):
        for feature in features:
            df[feature].fillna('', inplace=True)

    def preprocess_data_part(data_part):
        data_part = data_part.drop(auxiliary_columns, axis=1)
        
        fill_na(data_part, cat_features)
        fill_na(data_part, text_features)

        X = data_part.drop(['rating_10'], axis=1)
        y = data_part['rating_10']
        return X, y
    
    X_learn, y_learn = preprocess_data_part(learn)
    X_test, y_test = preprocess_data_part(test)

    return X_learn, X_test, y_learn, y_test

X_train, X_test, y_train, y_test = get_processed_rotten_tomatoes()

In [20]:
X_train.head(5)

,synopsis,rating_MPAA,genre,director,writer,box_office,runtime,studio,dvd_date_int,theater_date_int,review,fresh,critic,top_critic,publisher,date_int
0,A gay New Yorker stages a marriage of convenie...,R,Art House and International | Comedy | Drama |...,Ang Lee,Ang Lee | James Schamus | Neil Peng,NaN,111.0,,20040615,19930804,,fresh,Carol Cling,0,Las Vegas Review-Journal,20040416.0
1,Screenwriter Nimrod Antal makes an impressive ...,R,Action and Adventure | Art House and Internati...,,,116783.0,105.0,ThinkFilm Inc.,20050830,20050401,"One very long, dark ride.",rotten,,0,E! Online,20050422.0
2,"""Arctic Tale"" is an epic adventure that explor...",G,Documentary | Special Interest,Adam Ravetch | Sarah Robertson,Linda Woolverton | Mose Richards | Kristin Gore,598103.0,86.0,Paramount Vantage,20170801,20070817,I'm no holdout about the reality of global war...,rotten,Jack Mathews,1,New York Daily News,20070727.0
3,A dating doctor claims that with his services ...,PG-13,Comedy | Romance,Andrew Tennant | Andy Tennant,Kevin Bisch,177575142.0,120.0,Sony Pictures,20050614,20050211,... Adds up to far more than the formula typic...,fresh,Greg Maki,0,"Star-Democrat (Easton, MD)",20050211.0
4,R&amp;B; star Janet Jackson made an impressive...,R,Drama,John Singleton,John Singleton,NaN,104.0,,20010508,19930723,,fresh,Clint Morris,0,Film Threat,20050506.0


## Previously you can specify only categorical features and perform training like this:

In [21]:
X_train_no_text = X_train.drop(text_features, axis=1)
X_test_no_text = X_test.drop(text_features, axis=1)

learn_pool = Pool(
    X_train_no_text, 
    y_train, 
    cat_features=cat_features, 
    feature_names=list(X_train_no_text)
)
test_pool = Pool(
    X_test_no_text, 
    y_test, 
    cat_features=cat_features, 
    feature_names=list(X_train_no_text)
)

model = CatBoostClassifier(iterations=1000, learning_rate=0.03, eval_metric='Accuracy', task_type='GPU')
model.fit(learn_pool, eval_set=test_pool, verbose=50)

0:	learn: 0.3937699	test: 0.4022497	best: 0.4022497 (0)	total: 53.4ms	remaining: 53.3s
50:	learn: 0.4163304	test: 0.4213229	best: 0.4225455 (33)	total: 1.27s	remaining: 23.6s
100:	learn: 0.4280692	test: 0.4290256	best: 0.4298814 (97)	total: 2.47s	remaining: 22s
150:	learn: 0.4360174	test: 0.4325712	best: 0.4329380 (140)	total: 3.62s	remaining: 20.3s
200:	learn: 0.4423147	test: 0.4368505	best: 0.4372173 (182)	total: 4.78s	remaining: 19s
250:	learn: 0.4474505	test: 0.4374618	best: 0.4381954 (240)	total: 5.96s	remaining: 17.8s
300:	learn: 0.4526168	test: 0.4410075	best: 0.4411297 (299)	total: 7.13s	remaining: 16.6s
350:	learn: 0.4575385	test: 0.4413743	best: 0.4418633 (312)	total: 8.25s	remaining: 15.3s
400:	learn: 0.4627965	test: 0.4446754	best: 0.4446754 (399)	total: 9.38s	remaining: 14s
450:	learn: 0.4666178	test: 0.4456535	best: 0.4462648 (448)	total: 10.5s	remaining: 12.8s
500:	learn: 0.4697359	test: 0.4480988	best: 0.4482211 (498)	total: 11.6s	remaining: 11.6s
550:	learn: 0.4736794	

## Now you can specify also text features:

In [22]:
def fit_catboost_on_rotten_tomatoes(X_train, X_test, y_train, y_test, catboost_params={}, verbose=100):
    learn_pool = Pool(
        X_train, 
        y_train, 
        cat_features=cat_features,
        text_features=text_features,
        feature_names=list(X_train)
    )
    test_pool = Pool(
        X_test, 
        y_test, 
        cat_features=cat_features,
        text_features=text_features,
        feature_names=list(X_train)
    )
    
    catboost_default_params = {
        'iterations': 1000,
        'learning_rate': 0.03,
        'eval_metric': 'Accuracy',
        'task_type': 'GPU'
    }
    
    catboost_default_params.update(catboost_params)
    
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)

    return model

In [23]:
fit_catboost_on_rotten_tomatoes(X_train, X_test, y_train, y_test)

0:	learn: 0.4001895	test: 0.4054285	best: 0.4054285 (0)	total: 184ms	remaining: 3m 3s
100:	learn: 0.4465640	test: 0.4553124	best: 0.4553124 (100)	total: 15.6s	remaining: 2m 18s
200:	learn: 0.4576914	test: 0.4627705	best: 0.4628928 (197)	total: 28.1s	remaining: 1m 51s
300:	learn: 0.4645696	test: 0.4670498	best: 0.4670498 (285)	total: 39.3s	remaining: 1m 31s
400:	learn: 0.4725177	test: 0.4690060	best: 0.4691283 (399)	total: 50.4s	remaining: 1m 15s
500:	learn: 0.4791514	test: 0.4688837	best: 0.4703509 (448)	total: 1m 1s	remaining: 1m 1s
600:	learn: 0.4836146	test: 0.4685169	best: 0.4703509 (448)	total: 1m 13s	remaining: 48.5s
700:	learn: 0.4891477	test: 0.4703509	best: 0.4705954 (623)	total: 1m 24s	remaining: 35.9s
800:	learn: 0.4943140	test: 0.4699841	best: 0.4714513 (785)	total: 1m 35s	remaining: 23.7s
900:	learn: 0.4993886	test: 0.4707177	best: 0.4714513 (785)	total: 1m 46s	remaining: 11.7s
999:	learn: 0.5043103	test: 0.4709622	best: 0.4725517 (965)	total: 1m 56s	remaining: 0us
bestTes

# Accuracy was 0.4563 → currently it is 0.4707<br>Increased by 3% due to adding signals from text features

Also you can create Pool object with text features using file pathes. Then you need to specify all text features with type `Text` in column description file.

<span style="color:red;font-size:20px">Attention!</span>

<span style="font-size:18px">

1. Text features also cannot contain NaN values, so we converted them into strings manually.
2. Training with text features supported only on GPU.
3. The training may be performed only with classification losses and targets.
</span>

# How does it work?
## Text preprocessing

Usually we get our text as a sequence of Unicode symbols. So, if the task isn't a DNA classification we don't need such granularity, moreover, we need to extract more complicated entities, e.g. words. The process of extraction tokens -- words, numbers, punctuation symbols or special symbols which defines emoji from a sequence is called **tokenization**.<br>

Tokenization is the first part of text preprocessing in CatBoost and performed as a simple splitting a sequence on a string pattern (e.g. space).
### Example

In [24]:
text_small = [
    "Cats are so cute :)",
    "Mouse skare...",
    "The cat defeated the mouse",
    "Cute: Mice gather an army!",
    "Army of mice defeated the cat :(",
    "Cat offers peace",
    "Cat is skared :(",
    "Cat and mouse live in peace :)"
]

target_small = [1, 0, 1, 1, 0, 1, 0, 1]

In [25]:
from catboost.text_processing import Tokenizer

simple_tokenizer = Tokenizer()

def tokenize_texts(texts):
    return [simple_tokenizer.tokenize(text) for text in texts]

tokenized_text = tokenize_texts(text_small)
tokenized_text

[['Cats', 'are', 'so', 'cute', ':)'],
 ['Mouse', 'skare...'],
 ['The', 'cat', 'defeated', 'the', 'mouse'],
 ['Cute:', 'Mice', 'gather', 'an', 'army!'],
 ['Army', 'of', 'mice', 'defeated', 'the', 'cat', ':('],
 ['Cat', 'offers', 'peace'],
 ['Cat', 'is', 'skared', ':('],
 ['Cat', 'and', 'mouse', 'live', 'in', 'peace', ':)']]

## Punctuation handling, lowercasing, lemmatization

Lets take a closer look on the tokenization result of small text example -- the tokens contains a lot of mistakes:

1. They are glued with punctuation 'Cute:', 'army!', 'skare...'.
2. The words 'Cat' and 'cat', 'Mice' and 'mice' seems to have same meaning, perhaps they should be the same tokens.
3. The same problem with tokens 'are'/'is' -- they are inflected forms of same token 'be'.

**Punctuation handling** and **lemmatization** processes help to overcome these problems.

### Punctuation handling

Depending on the task, punctuation handling process may:
1. Completely remove all punctuation.
2. Escape it with spaces.
3. Just left as is (e.g. for more complicated tokens collection).

### Examples

In [26]:
tokenizer = Tokenizer(
    lowercasing=True,
    separator_type='BySense',
    token_types=['Word', 'Number', 'Punctuation']
)

text_small_spaced = [' '.join(tokenizer.tokenize(text)) for text in text_small]
text_small_spaced

['cats are so cute :)',
 'mouse skare ...',
 'the cat defeated the mouse',
 'cute : mice gather an army !',
 'army of mice defeated the cat :(',
 'cat offers peace',
 'cat is skared :(',
 'cat and mouse live in peace :)']

### Removing stop words

**Stop words** -- the words that are considered to be uninformative in this task, e.g. function words such as *the, is, at, which, on*.<br>
Usually stop words are removed during text preprocessing to reduce the amount of information that is considered for further algorithms.<br>
Stop words are collected manually (in dictionary form) or automatically, for example taking the most frequent words.

In [27]:
stop_words = ['be', 'is', 'are', 'the', 'an', 'of', 'and', 'in']

def remove_words(texts, words):
    texts_copy = []
    words_set = set(words)

    for text in tokenize_texts(texts):
        text_copy = []
        for token in text:
            if token not in words_set:
                text_copy.append(token)
        texts_copy.append(' '.join(text_copy))
            
    return texts_copy
    
text_small_no_stop = remove_words(text_small_spaced, stop_words)
text_small_no_stop

['cats so cute :)',
 'mouse skare ...',
 'cat defeated mouse',
 'cute : mice gather army !',
 'army mice defeated cat :(',
 'cat offers peace',
 'cat skared :(',
 'cat mouse live peace :)']

### Lemmatization

Lemma (Wikipedia) -- is the canonical form, dictionary form, or citation form of a set of words.<br>
For example, the lemma "go" represents the inflected forms "go", "goes", "going", "went", and "gone".<br>
The process of convertation word to its lemma called **lemmatization**.

In [28]:
from pattern.en import lemma

def lemmatize_text(text):
    return " ".join([lemma(word) for word in text.decode('utf-8').split()])

def lemmatize_texts(texts):
    return [lemmatize_text(text) for text in texts]

text_small_lemmatized = lemmatize_texts(text_small_no_stop)
text_small_lemmatized = tokenize_texts(text_small_lemmatized)
text_small_lemmatized

[['cat', 'so', 'cute', ':)'],
 ['mouse', 'skare', '...'],
 ['cat', 'defeat', 'mouse'],
 ['cute', ':', 'mice', 'gather', 'army', '!'],
 ['army', 'mice', 'defeat', 'cat', ':('],
 ['cat', 'offer', 'peace'],
 ['cat', 'skare', ':('],
 ['cat', 'mouse', 'live', 'peace', ':)']]

Now words with same meaning represented by the same token, tokens are not glued with punctuation.

<span style="color:red">Be carefull.</span> You should verify for your own task:<br>
Is it realy necessary to remove punctuation, lowercasing sentences or performing a lemmatization and/or by word tokenization?<br>

There is still problems with tokens 'mice'/'mouse', gensim lemmatizer can handle it, we provide code in the cell below, but for the sake of simplicity and speed in this tutorial we will use the first one.

In [29]:
from gensim.utils import lemmatize

def lemmatize_text_gensim(text):
    result = []
    for token in simple_tokenizer.tokenize(text):
        lemmas = lemmatize(token)
        if len(lemmas) == 0:
            lemma = token.lower()
        else:
            lemma = lemmas[0].decode('utf-8').split('/')[0]
            
        result.append(lemma)
    return ' '.join(result)

## Let's check up accuracy with new text preprocessing

Since CatBoost doesn't perform spacing punctuation, lowercasing letters and lemmatization, we need to preprocess text manually and then pass it to learning algorithm.

Since the natural text features is only synopsis and review, we will preprocess only them.

In [30]:
def preprocess_data(X):
    X_preprocessed = X.copy()
    for feature in ['synopsis', 'review']:
        X_preprocessed[feature] = X[feature].apply(lambda x: lemmatize_text(' '.join(tokenizer.tokenize(x))))
    return X_preprocessed

X_preprocessed_train = preprocess_data(X_train)
X_preprocessed_test = preprocess_data(X_test)

In [31]:
X_preprocessed_train['synopsis'].head(10)

0    a gay new yorker stage a marriage of convenien...
1    screenwriter nimrod antal make an impressive d...
2    " arctic tale " be an epic adventure that expl...
3    a date doctor claim that with hi service he ca...
4    r & amp ; b ; star janet jackson make an impre...
5    for four year , the courageou crew of the nsea...
6    thi holiday season , acclaim filmmaker cameron...
7    three billboard outside ebb , missouri be a da...
8    alfie elkin be a philosophical womanizer who b...
9    george cukor' remake of the 1940 film gaslight...
Name: synopsis, dtype: object

In [32]:
fit_catboost_on_rotten_tomatoes(X_preprocessed_train, X_preprocessed_test, y_train, y_test)

0:	learn: 0.4000367	test: 0.4045727	best: 0.4045727 (0)	total: 181ms	remaining: 3m
100:	learn: 0.4474810	test: 0.4549456	best: 0.4549456 (100)	total: 15.4s	remaining: 2m 16s
200:	learn: 0.4596784	test: 0.4611811	best: 0.4615479 (195)	total: 28s	remaining: 1m 51s
300:	learn: 0.4677183	test: 0.4661939	best: 0.4669275 (287)	total: 39.5s	remaining: 1m 31s
400:	learn: 0.4745659	test: 0.4668052	best: 0.4675388 (394)	total: 51s	remaining: 1m 16s
500:	learn: 0.4830949	test: 0.4708400	best: 0.4710845 (445)	total: 1m 2s	remaining: 1m 2s
600:	learn: 0.4893006	test: 0.4730407	best: 0.4742634 (586)	total: 1m 13s	remaining: 49.1s
700:	learn: 0.4949865	test: 0.4735298	best: 0.4746302 (696)	total: 1m 25s	remaining: 36.4s
800:	learn: 0.5013145	test: 0.4741411	best: 0.4752415 (779)	total: 1m 36s	remaining: 23.9s
900:	learn: 0.5069393	test: 0.4743856	best: 0.4752415 (779)	total: 1m 47s	remaining: 11.8s
999:	learn: 0.5115248	test: 0.4741411	best: 0.4754860 (934)	total: 1m 58s	remaining: 0us
bestTest = 0.4

## Dictionary

After the first stage, preprocessing of text and tokenization, the second stage starts. The second stage uses the prepared text to select a set of units, which will be used for building new numerical features.

A set of selected units is called dictionary. It might contain words, word bigramms, or character n-gramms.

### Examples
Lets build a dictionary for our small text example:

In [19]:
def build_dictionary(tokenized_texts):
    dictionary = {}

    for text in tokenized_texts:
        for token in text:
            if token not in dictionary:
                size = len(dictionary)
                dictionary[token] = size

    return dictionary

def print_dictionary(dictionary, n_items=5):
    dict_items = sorted(dictionary.items(), key=lambda x: x[1])

    for i in range(n_items):
        word, word_id = dict_items[i]
        print('word="{}" has id="{}"'.format(word, word_id))
    
    print('...')

dictionary = build_dictionary(text_small_lemmatized)
print_dictionary(dictionary)

word="cat" has id="0"
word="so" has id="1"
word="cute" has id="2"
word=":)" has id="3"
word="mouse" has id="4"
...


## Convertation into fixed size vectors

The majority of classic ML algorithms are computing and performing predictions on a fixed number of features $F$.<br>
That means that learning set $X = \{x_i\}$ contains vectors $x_i = (a_0, a_1, ..., a_F)$ where $F$ is constant.

Since text object $x$ is not a fixed length vector, we need to perform preprocessing of the origin set $D$.<br>
One of the simplest text to vector encoding technique is **Bag of words (BoW)**.

### Bag of words algorithm

The algorithm takes in a dictionary and a text.<br>
During the algorithm text $x = (a_0, a_1, ..., a_k)$ converted into vector $\tilde x = (b_0, b_1, ..., b_F)$,<br> where $b_i$ is number of occurences word with id=$i$ from dictionary into text $x$.

In [20]:
def bag_of_words(texts, dictionary):
    encoded_vectors = []
    dictionary_size = len(dictionary)

    for text in texts:
        vector = [0] * dictionary_size

        for token in text:
            if token in dictionary:
                token_id = dictionary[token]
                vector[token_id] = 1
    
        encoded_vectors.append(vector)
    
    return encoded_vectors

def print_bow_features(bag_of_words, dictionary):
    sorted_dict = sorted(dictionary.items(), key=lambda x: x[1])
    keys = [x[0] for x in sorted_dict]
    bow_df = pd.DataFrame(data=bag_of_words, columns=keys)
    print(bow_df)
    

bow_features = bag_of_words(text_small_lemmatized, dictionary)

print_bow_features(bow_features, dictionary)

   cat  so  cute  :)  mouse  skare  ...  defeat  :  mice  gather  army  !  :(  \
0    1   1     1   1      0      0    0       0  0     0       0     0  0   0   
1    0   0     0   0      1      1    1       0  0     0       0     0  0   0   
2    1   0     0   0      1      0    0       1  0     0       0     0  0   0   
3    0   0     1   0      0      0    0       0  1     1       1     1  1   0   
4    1   0     0   0      0      0    0       1  0     1       0     1  0   1   
5    1   0     0   0      0      0    0       0  0     0       0     0  0   0   
6    1   0     0   0      0      1    0       0  0     0       0     0  0   1   
7    1   0     0   1      1      0    0       0  0     0       0     0  0   0   

   offer  peace  live  
0      0      0     0  
1      0      0     0  
2      0      0     0  
3      0      0     0  
4      0      0     0  
5      1      1     0  
6      0      0     0  
7      0      1     1  


For example, having such vectors we can fit Linear or Naive bayes model

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix

def fit_linear_model(X, c):
    model = LogisticRegression()
    model.fit(X, c)
    return model

def fit_naive_bayes(X, c):
    clf = MultinomialNB()
    if isinstance(X, csr_matrix):
        X.eliminate_zeros()
    clf.fit(X, c)
    return clf

linear_model = fit_linear_model(bow_features, target_small)
naive_bayes = fit_naive_bayes(bow_features, target_small)

/home/d-kruchinin/.local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [22]:
from sklearn.metrics import log_loss

def evaluate_model_logloss(model, X, c):
    c_pred = model.predict_proba(X)[:,1]
    metric = log_loss(c, c_pred)
    print('Logloss: ' + str(metric))

print('Linear model')
evaluate_model_logloss(linear_model, bow_features, target_small)
print('Naive bayes')
evaluate_model_logloss(naive_bayes, bow_features, target_small)
print('Comparing to constant prediction')
logloss_constant_prediction = log_loss(target_small, np.ones(shape=(len(text_small), 2)) * 0.5)
print('Logloss: ' + str(logloss_constant_prediction))

Linear model
Logloss: 0.3314294362422291
Naive bayes
Logloss: 0.1667380176962438
Comparing to constant prediction
Logloss: 0.6931471805599453


## Looking at sequences of letters / words

Let's look at the example: texts 'The cat defeated the mouse' and 'Army of mice defeated the cat :('<br>
Simplifying it we have three tokens in each sentence 'cat defeat mouse' and 'mouse defeat cat'.<br>
After applying BoW we get two equal vectors with the opposite meaning:

| cat | mouse | defeat |
|-----|-------|--------|
| 1   | 1     | 1      |
| 1   | 1     | 1      |

How to distinguish them?
Lets add sequences of words as a single tokens into our dictionary:

| cat | mouse | defeat | cat_defeat | mouse_defeat | defeat_cat | defeat_mouse |
|-----|-------|--------|------------|--------------|------------|--------------|
| 1   | 1     | 1      | 1          | 0            | 0          | 1            |
| 1   | 1     | 1      | 0          | 1            | 1          | 0            |

**N-gram** is a continguous sequence of $n$ items from a given sample of text or speech (Wikipedia).<br>
In example above Bi-gram (Bigram) = 2-gram of words.

Ngrams help to add into vectors more information about text structure, moreover there are n-grams has no meanings in separation, for example, 'Mickey Mouse company'.

## Examples

In [23]:
def build_bigram_dictionary(tokenized_texts):
    dictionary = {}
    for text in tokenized_texts:
        for i in range(len(text) - 1):
            token1, token2 = text[i], text[i + 1]
            bigram = token1 + ' ' + token2
            
            if bigram not in dictionary:
                dictionary_size = len(dictionary)
                dictionary[bigram] = dictionary_size

    return dictionary

bigram_word_dictionary = build_bigram_dictionary(text_small_lemmatized)
print_dictionary(bigram_word_dictionary)

word="cat so" has id="0"
word="so cute" has id="1"
word="cute :)" has id="2"
word="mouse skare" has id="3"
word="skare ..." has id="4"
...


## Dictionaries in CatBoost

To specify which type of dictionary to create in CatBoost you need to pass `dictionaries` parameter. This parameter specifies all dictionaries that are computed during text preprocessing procedure.

Dictionaries parameters specified as list of strings, each string is a description of dictionary in the following format:<br>
``'DictionaryName:[Param1=Value1,[Param2=Value2]]'``

Here is a list of all parameters:<br>
``min_token_occurrence`` -- number; minimal token occurence to enter to dictionary<br>
``max_dict_size`` -- number; maximum dictionary size<br>
``token_level_type`` -- string: ``Word`` or ``Letter``; use letter or word tokens<br>
``gram_order`` -- number; build n-gram dictionary.

### Let's see how these parameters affect the quality

``min_token_occurrence`` -- parameter can be very useful for filtering too rare tokens, this helps to avoid overfitting<br>
``max_dict_size`` -- parameter can help to control the size of model

In [24]:
fit_catboost_on_rotten_tomatoes(
    X_preprocessed_train,
    X_preprocessed_test, 
    y_train, 
    y_test,
    catboost_params={
        'dictionaries': [
            'Word:min_token_occurrence=5',
            'BiGram:gram_order=2'
        ],
        'text_processing': [
            'NaiveBayes+Word|BoW+Word,BiGram'
        ]
    }
)

0:	learn: 0.3855466	test: 0.3940580	best: 0.3940580 (0)	total: 107ms	remaining: 1m 46s
100:	learn: 0.4497432	test: 0.4521335	best: 0.4529894 (97)	total: 4.49s	remaining: 39.9s
200:	learn: 0.4622463	test: 0.4624037	best: 0.4637486 (189)	total: 8.5s	remaining: 33.8s
300:	learn: 0.4705307	test: 0.4636264	best: 0.4639932 (299)	total: 12.5s	remaining: 29.1s
400:	learn: 0.4780509	test: 0.4653381	best: 0.4671720 (339)	total: 16.6s	remaining: 24.8s
500:	learn: 0.4839203	test: 0.4666830	best: 0.4680279 (466)	total: 20.7s	remaining: 20.6s
600:	learn: 0.4906151	test: 0.4702286	best: 0.4707177 (592)	total: 24.9s	remaining: 16.5s
700:	learn: 0.4963928	test: 0.4703509	best: 0.4714513 (651)	total: 29.2s	remaining: 12.5s
800:	learn: 0.5022316	test: 0.4724294	best: 0.4729184 (795)	total: 33.6s	remaining: 8.34s
900:	learn: 0.5072450	test: 0.4740188	best: 0.4746302 (882)	total: 38s	remaining: 4.17s
999:	learn: 0.5120751	test: 0.4749969	best: 0.4758528 (946)	total: 42.2s	remaining: 0us
bestTest = 0.475852

## Feature calculation in CatBoost

Since the text is converted into a sequence of token indices, this information allows CatBoost to compute different numeric features:

1. Bag of words: 0/1 features (text sample has or not token_id), number of produced numeric features = dictionary size.
2. NaiveBayes: [Multinomial naive bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) model, produce number of features equal to number of classes.<br> To avoid target leakage this model is computed online on a several dataset permutations like we do with [estimation of CTRs](https://catboost.ai/docs/concepts/algorithm-main-stages_cat-to-numberic.html).
3. [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) also computed online, it's a function that used for ranking purposes by search engines to estimate the relevance of documents.

You can specify features to compute in parameter ``text_processing``.

## Parameter ``text_processing``

Parameter **text_processing** specifies how text features are preprocessed.

Text processing parameters specified as a list of strings, each string is a description of feature preprocessing in the following format:<br>
``'FeatureId~[FeatureEstimator1+DictionaryName1[|FeatureEstimator2+DictionaryName2]]'``

Example: ``'0~BoW+Word|NaiveBayes+Word,Bigram'``,
this means that on 0-th text feature ``BoW`` and ``NaiveBayes`` features will be computed using ``Word`` and ``Word,Bigram`` dictionaries correspondingly.<br>
Also it may be specified ``default~...`` text feature (or empty FeatureId), it means that all text features will be preprocessed with the same procedure, specified in parameters.

Dictionaries names are taken from ``dictionaries`` parameter.

Also you can specify parameters for estimators, e.g. for Bag of words you may specify parameter ``top_tokens_count``, this parameter sets maximum number of tokens that will be used for vectorization in bag of words, the most frequent $n$ tokens are taken. Parameter ``top_tokens_count`` **highly affect both on CPU ang GPU RAM usage** in BoW estimator.

In [25]:
fit_catboost_on_rotten_tomatoes(
    X_preprocessed_train,
    X_preprocessed_test, 
    y_train, 
    y_test,
    catboost_params={
        'dictionaries': [
            'Word:min_token_occurrence=5',
            'BiGram:gram_order=2'
        ],
        'text_processing': [
            'NaiveBayes+Word|BoW:top_tokens_count=1000+Word,BiGram|BM25+Word'
        ]
    }
)

0:	learn: 0.3985388	test: 0.4054285	best: 0.4054285 (0)	total: 94.5ms	remaining: 1m 34s
100:	learn: 0.4494987	test: 0.4534784	best: 0.4534784 (100)	total: 4.22s	remaining: 37.5s
200:	learn: 0.4620934	test: 0.4587358	best: 0.4593471 (196)	total: 8.16s	remaining: 32.5s
300:	learn: 0.4711727	test: 0.4635041	best: 0.4639932 (265)	total: 12.1s	remaining: 28.1s
400:	learn: 0.4797628	test: 0.4675388	best: 0.4681501 (399)	total: 16.1s	remaining: 24s
500:	learn: 0.4874052	test: 0.4691283	best: 0.4696173 (496)	total: 20.1s	remaining: 20s
600:	learn: 0.4942834	test: 0.4710845	best: 0.4719403 (596)	total: 24s	remaining: 15.9s
700:	learn: 0.5011617	test: 0.4709622	best: 0.4719403 (596)	total: 27.8s	remaining: 11.9s
800:	learn: 0.5069393	test: 0.4708400	best: 0.4719403 (596)	total: 31.8s	remaining: 7.89s
900:	learn: 0.5133284	test: 0.4716958	best: 0.4723071 (836)	total: 35.7s	remaining: 3.92s
999:	learn: 0.5190144	test: 0.4721849	best: 0.4732852 (950)	total: 39.6s	remaining: 0us
bestTest = 0.4732852

# Summary: Text features in CatBoost

### The algorithm:
1. Input text is loaded as a usual column. ``text_column: [string]``.
2. Each text sample is tokenized via splitting by space. ``tokenized_column: [[string]]``.
3. Dictionary estimation.
4. Each string in tokenized column is converted into token_id from dictionary. ``text: [[token_id]]``.
5. Depending on the parameters CatBoost produce features basing on the resulting text column: Bag of words, Multinomial naive bayes or Bm25.
6. Computed float features are passed into the usual CatBoost learning algorithm.

## Summarizing table:

| method description                                                | Accuracy |
|-------------------------------------------------------------------|----------|
| Without text features                                             | 0.4562   |
| With unpreprocessed text features                                 | 0.4707   |
| After punctuation handling and lemmatization (only review column) | 0.4719   |
| After adding bigrams                                              | 0.4759   |

## Simplified comparison with classic methods: Naive Bayes and Logistic regression

Let's take only one text column to compare quality of text classification.<br>

In [26]:
X_train_one_column = pd.DataFrame(X_preprocessed_train['review'])
X_test_one_column = pd.DataFrame(X_preprocessed_test['review'])

In [27]:
def fit_catboost_one_column(X_train, X_test, y_train, y_test, catboost_params={}, verbose=0):
    learn_pool = Pool(X_train, y_train, text_features=[0])
    test_pool = Pool(X_test, y_test, text_features=[0])
    
    catboost_default_params = {
        'iterations': 1000,
        'learning_rate': 0.03,
        'eval_metric': 'Accuracy',
        'task_type': 'GPU'
    }
    
    catboost_default_params.update(catboost_params)
    
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)

    return model

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

def vectorize(X, params):
    vectorizer = CountVectorizer(**params)
    vectorizer.fit(X)
    return vectorizer.transform(X), vectorizer

def eval_accuracy(model, X, c):
    c_pred = model.predict(X)
    return accuracy_score(c_pred, c)

def fit_and_compute_accuracy(X_train, X_test, y_train, y_test, vectorizer_params={}, catboost_params={}):
    X_train_bow, vectorizer = vectorize(X_train.iloc[:,0], vectorizer_params)
    X_test_bow = vectorizer.transform(X_test.iloc[:,0])
    
    print('fitting linear model')
    linear_model = fit_linear_model(X_train_bow, y_train)
    
    print('fitting naive bayes model')
    naive_bayes = fit_naive_bayes(X_train_bow, y_train)
    
    print('fitting catboost model')
    cb_model = fit_catboost_one_column(X_train, X_test, y_train, y_test, catboost_params)

    linear_accuracy = eval_accuracy(linear_model, X_test_bow, y_test)
    naive_bayes_accuracy = eval_accuracy(naive_bayes, X_test_bow, y_test)
    cb_accuracy = eval_accuracy(cb_model, X_test, y_test)

    results = pd.DataFrame(
        data=[linear_accuracy, naive_bayes_accuracy, cb_accuracy], 
        index=['Linear model', 'Naive bayes', 'CatBoost'],
        columns=['Accuracy']
    )
    print(results)

### Experiment without bigrams

In [29]:
fit_and_compute_accuracy(
    X_train_one_column, 
    X_test_one_column, 
    y_train, 
    y_test,
    catboost_params = {
        'dictionaries': ['Word:token_level_type=Word,min_token_occurrence=5'],
        'text_processing': ['NaiveBayes+Word|BoW+Word']
    }
)

fitting linear model


/home/d-kruchinin/.local/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


fitting naive bayes model
fitting catboost model
              Accuracy
Linear model  0.292945
Naive bayes   0.301871
CatBoost      0.325223


### Experiment with bigrams

In [30]:
fit_and_compute_accuracy(
    X_train_one_column,
    X_test_one_column, 
    y_train, 
    y_test, 
    vectorizer_params = {'ngram_range': (1, 2)},
    catboost_params = {
        'dictionaries': [
            'Word:token_level_type=Word,min_token_occurrence=5', 
            'BiGram:gram_order=2,min_token_occurrence=4'
        ],
        'text_processing': ['NaiveBayes+Word,BiGram|BoW+Word,BiGram']
    }
)

fitting linear model
fitting naive bayes model
fitting catboost model
              Accuracy
Linear model  0.302604
Naive bayes   0.295146
CatBoost      0.329747


| method description      | Linear model | Naive bayes | CatBoost |
|-------------------------|--------------|-------------|----------|
| Without bigrams         | 0.2929       | 0.3019      | 0.3252   |
| With bigrams            | 0.3026       | 0.2951      | 0.3294   |

